# レコメンドAIの数理 - Part 1: 数学の基礎

このNotebookでは、レコメンドシステムを理解するために必要な数学的基礎を学びます。

## 目次
1. 三角関数とcos θ
2. ベクトルと内積
3. コサイン類似度

## 環境設定

In [ ]:
# 日本語フォントのインストール（Google Colab用）
!pip install japanize-matplotlib -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

# グラフの設定
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 12

---
# セクション1: 三角関数とcos θ

## 単位円での定義

三角関数は**単位円**（半径1の円）を使って定義されます。

単位円上の点 $(x, y)$ において、原点からその点への線分がx軸となす角度を $\theta$ とすると：

$$\cos\theta = x \quad (x座標)$$
$$\sin\theta = y \quad (y座標)$$

### cos θの意味

- $\cos\theta = 1$：角度が0°（同じ方向）
- $\cos\theta = 0$：角度が90°（直交）
- $\cos\theta = -1$：角度が180°（反対方向）

つまり、**cos θは2つの方向がどれだけ似ているかを表す指標**です。

In [ ]:
# 単位円の可視化
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

# 単位円を描画
theta_circle = np.linspace(0, 2*np.pi, 100)
ax.plot(np.cos(theta_circle), np.sin(theta_circle), 'b-', linewidth=2, label='単位円')

# 軸を描画
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)

# いくつかの角度での点を表示
angles = [0, 30, 45, 60, 90, 120, 180]
colors = ['red', 'orange', 'gold', 'green', 'blue', 'purple', 'brown']

for angle, color in zip(angles, colors):
    rad = np.radians(angle)
    x, y = np.cos(rad), np.sin(rad)
    
    # 点を描画
    ax.plot(x, y, 'o', color=color, markersize=10)
    
    # 原点から点への線
    ax.plot([0, x], [0, y], '-', color=color, linewidth=2)
    
    # ラベル
    ax.annotate(f'{angle}°\ncos={x:.2f}', 
                xy=(x, y), 
                xytext=(x*1.3, y*1.3),
                fontsize=10,
                ha='center')

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_aspect('equal')
ax.set_xlabel('x (= cos θ)')
ax.set_ylabel('y (= sin θ)')
ax.set_title('単位円とcos θの関係')
ax.grid(True, alpha=0.3)
plt.show()

In [ ]:
# 角度とcos値の関係グラフ
fig, ax = plt.subplots(figsize=(10, 5))

theta_deg = np.linspace(0, 180, 100)
theta_rad = np.radians(theta_deg)
cos_values = np.cos(theta_rad)

ax.plot(theta_deg, cos_values, 'b-', linewidth=2)

# 重要な点をマーク
important_angles = [0, 90, 180]
for angle in important_angles:
    cos_val = np.cos(np.radians(angle))
    ax.plot(angle, cos_val, 'ro', markersize=10)
    ax.annotate(f'({angle}°, {cos_val:.0f})', 
                xy=(angle, cos_val),
                xytext=(angle+10, cos_val+0.1),
                fontsize=12)

ax.axhline(y=0, color='k', linewidth=0.5, linestyle='--')
ax.set_xlabel('角度 θ (度)')
ax.set_ylabel('cos θ')
ax.set_title('角度とcos θの関係')
ax.set_xticks([0, 30, 60, 90, 120, 150, 180])
ax.grid(True, alpha=0.3)

# 解釈を追加
ax.text(45, 0.5, '似ている\n(cos θ > 0)', ha='center', fontsize=11, color='green')
ax.text(135, -0.5, '異なる\n(cos θ < 0)', ha='center', fontsize=11, color='red')

plt.tight_layout()
plt.show()

---
# セクション2: ベクトルと内積

## ベクトルとは

ベクトルは**大きさと方向を持つ量**です。数学的には、数値の順序付きリストとして表現します。

$$\vec{a} = \begin{pmatrix} a_1 \\ a_2 \\ \vdots \\ a_n \end{pmatrix}$$

## ノルム（大きさ）

ベクトルの**ノルム**（長さ、大きさ）は以下で計算します：

$$\|\vec{a}\| = \sqrt{a_1^2 + a_2^2 + \cdots + a_n^2} = \sqrt{\sum_{i=1}^{n} a_i^2}$$

## 内積（ドット積）

2つのベクトルの**内積**は以下で定義されます：

$$\vec{a} \cdot \vec{b} = a_1 b_1 + a_2 b_2 + \cdots + a_n b_n = \sum_{i=1}^{n} a_i b_i$$

### 内積の幾何学的意味

内積は次の式でも表せます：

$$\vec{a} \cdot \vec{b} = \|\vec{a}\| \|\vec{b}\| \cos\theta$$

ここで $\theta$ は2つのベクトルのなす角度です。

In [ ]:
# ベクトルの基本操作

# 2次元ベクトルの定義
a = np.array([3, 4])
b = np.array([1, 2])

print("=== ベクトルの基本 ===")
print(f"ベクトル a = {a}")
print(f"ベクトル b = {b}")
print()

# ノルムの計算
norm_a = np.linalg.norm(a)
norm_b = np.linalg.norm(b)
print(f"||a|| = √(3² + 4²) = √{3**2 + 4**2} = {norm_a}")
print(f"||b|| = √(1² + 2²) = √{1**2 + 2**2} = {norm_b:.4f}")
print()

# 内積の計算
dot_product = np.dot(a, b)
print(f"a · b = 3×1 + 4×2 = {3*1} + {4*2} = {dot_product}")

In [ ]:
# 2次元ベクトルの可視化
fig, ax = plt.subplots(figsize=(8, 8))

# ベクトルを矢印で描画
ax.quiver(0, 0, a[0], a[1], angles='xy', scale_units='xy', scale=1, 
          color='blue', width=0.02, label=f'a = {a}')
ax.quiver(0, 0, b[0], b[1], angles='xy', scale_units='xy', scale=1, 
          color='red', width=0.02, label=f'b = {b}')

# 角度を計算して表示
cos_theta = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
theta = np.arccos(cos_theta)
theta_deg = np.degrees(theta)

# 角度の弧を描画
arc_theta = np.linspace(np.arctan2(b[1], b[0]), np.arctan2(a[1], a[0]), 30)
arc_r = 0.8
ax.plot(arc_r * np.cos(arc_theta), arc_r * np.sin(arc_theta), 'g-', linewidth=2)
ax.text(1.0, 1.0, f'θ = {theta_deg:.1f}°', fontsize=12, color='green')

ax.set_xlim(-1, 5)
ax.set_ylim(-1, 5)
ax.set_aspect('equal')
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)
ax.grid(True, alpha=0.3)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('2次元ベクトルの可視化')
ax.legend(loc='upper left')

plt.show()

print(f"\n2つのベクトルのなす角 θ = {theta_deg:.2f}°")
print(f"cos θ = {cos_theta:.4f}")

In [ ]:
# 内積計算関数
def dot_product_manual(a, b):
    """
    2つのベクトルの内積を計算する
    
    Parameters:
    -----------
    a : array-like
        1つ目のベクトル
    b : array-like
        2つ目のベクトル
    
    Returns:
    --------
    float
        内積の値
    """
    a = np.array(a)
    b = np.array(b)
    
    if len(a) != len(b):
        raise ValueError("ベクトルの次元が一致しません")
    
    result = 0
    for i in range(len(a)):
        result += a[i] * b[i]
    
    return result

# テスト
print("=== 内積計算のテスト ===")
v1 = [1, 2, 3]
v2 = [4, 5, 6]
print(f"v1 = {v1}")
print(f"v2 = {v2}")
print(f"手動計算: v1 · v2 = 1×4 + 2×5 + 3×6 = {1*4} + {2*5} + {3*6} = {dot_product_manual(v1, v2)}")
print(f"NumPy: np.dot(v1, v2) = {np.dot(v1, v2)}")

---
# セクション3: コサイン類似度

## コサイン類似度の公式

内積の幾何学的定義から：

$$\vec{a} \cdot \vec{b} = \|\vec{a}\| \|\vec{b}\| \cos\theta$$

これを変形すると：

$$\cos\theta = \frac{\vec{a} \cdot \vec{b}}{\|\vec{a}\| \|\vec{b}\|} = \frac{\sum_{i=1}^{n} a_i b_i}{\sqrt{\sum_{i=1}^{n} a_i^2} \cdot \sqrt{\sum_{i=1}^{n} b_i^2}}$$

この値を**コサイン類似度**と呼びます。

## 値の解釈

| コサイン類似度 | 意味 |
|:---:|:---|
| 1 | 完全に同じ方向（最も類似） |
| 0 | 直交（無関係） |
| -1 | 完全に反対方向（最も非類似） |

## なぜコサイン類似度が有用なのか？

コサイン類似度は**ベクトルの大きさに依存しない**という特徴があります。

例えば、ユーザーAが映画を「5, 4, 5」と評価し、ユーザーBが「10, 8, 10」と評価した場合、
スケールは異なりますが、好みの傾向は同じです。コサイン類似度はこれを正しく捉えます。

In [ ]:
def cosine_similarity(a, b):
    """
    2つのベクトルのコサイン類似度を計算する
    
    Parameters:
    -----------
    a : array-like
        1つ目のベクトル
    b : array-like
        2つ目のベクトル
    
    Returns:
    --------
    float
        コサイン類似度（-1から1の値）
    """
    a = np.array(a)
    b = np.array(b)
    
    # 内積
    dot = np.dot(a, b)
    
    # ノルム
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    # ゼロベクトルのチェック
    if norm_a == 0 or norm_b == 0:
        return 0.0
    
    return dot / (norm_a * norm_b)

# テスト
print("=== コサイン類似度の計算 ===")
print()

# 同じ方向のベクトル
v1 = [1, 2, 3]
v2 = [2, 4, 6]  # v1の2倍
print(f"v1 = {v1}")
print(f"v2 = {v2} (v1の2倍)")
print(f"コサイン類似度 = {cosine_similarity(v1, v2):.4f}")
print("→ 同じ方向なので1.0")
print()

In [ ]:
# 様々なケースでのコサイン類似度
print("=== 様々なケースでのコサイン類似度 ===")
print()

# ケース1: 直交するベクトル
v1 = [1, 0]
v2 = [0, 1]
print(f"直交: {v1} と {v2}")
print(f"  コサイン類似度 = {cosine_similarity(v1, v2):.4f} (直交 = 無関係)")
print()

# ケース2: 反対方向
v1 = [1, 2]
v2 = [-1, -2]
print(f"反対方向: {v1} と {v2}")
print(f"  コサイン類似度 = {cosine_similarity(v1, v2):.4f} (反対 = 最も非類似)")
print()

# ケース3: 似ているベクトル
v1 = [5, 4, 5]
v2 = [4, 5, 4]
print(f"似ている: {v1} と {v2}")
print(f"  コサイン類似度 = {cosine_similarity(v1, v2):.4f}")
print()

# ケース4: レコメンドでの例（映画評価）
print("=== レコメンドシステムでの例 ===")
print("映画評価: [アクション, コメディ, ホラー]")
user_a = [5, 1, 4]  # アクションとホラーが好き
user_b = [4, 2, 5]  # 似た好み
user_c = [1, 5, 1]  # コメディが好き

print(f"ユーザーA: {user_a}")
print(f"ユーザーB: {user_b}")
print(f"ユーザーC: {user_c}")
print()
print(f"AとBの類似度: {cosine_similarity(user_a, user_b):.4f} (似た好み)")
print(f"AとCの類似度: {cosine_similarity(user_a, user_c):.4f} (異なる好み)")

In [ ]:
# 2次元でのコサイン類似度の可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

cases = [
    ([1, 0.5], [0.8, 0.4], "似ている"),
    ([1, 0], [0, 1], "直交"),
    ([1, 1], [-1, -1], "反対")
]

for ax, (v1, v2, title) in zip(axes, cases):
    v1, v2 = np.array(v1), np.array(v2)
    sim = cosine_similarity(v1, v2)
    
    # ベクトルを描画
    ax.quiver(0, 0, v1[0], v1[1], angles='xy', scale_units='xy', scale=1,
              color='blue', width=0.03, label=f'a = {list(v1)}')
    ax.quiver(0, 0, v2[0], v2[1], angles='xy', scale_units='xy', scale=1,
              color='red', width=0.03, label=f'b = {list(v2)}')
    
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)
    ax.grid(True, alpha=0.3)
    ax.set_title(f'{title}\ncos類似度 = {sim:.2f}')
    ax.legend(loc='upper left', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# 3次元での計算例
print("=== 3次元ベクトルでのコサイン類似度計算 ===")
print()

# 映画の評価値（3つの映画）
print("3つの映画に対する評価値:")
print("  映画: [映画A, 映画B, 映画C]")
print()

user1 = np.array([5, 3, 1])
user2 = np.array([4, 4, 2])
user3 = np.array([1, 2, 5])

print(f"ユーザー1: {user1}")
print(f"ユーザー2: {user2}")
print(f"ユーザー3: {user3}")
print()

# 計算過程を詳しく表示
print("--- ユーザー1とユーザー2の類似度計算 ---")
dot = np.dot(user1, user2)
norm1 = np.linalg.norm(user1)
norm2 = np.linalg.norm(user2)

print(f"内積: {user1[0]}×{user2[0]} + {user1[1]}×{user2[1]} + {user1[2]}×{user2[2]}")
print(f"     = {user1[0]*user2[0]} + {user1[1]*user2[1]} + {user1[2]*user2[2]} = {dot}")
print(f"||user1|| = √({user1[0]}² + {user1[1]}² + {user1[2]}²) = √{sum(user1**2)} = {norm1:.4f}")
print(f"||user2|| = √({user2[0]}² + {user2[1]}² + {user2[2]}²) = √{sum(user2**2)} = {norm2:.4f}")
print(f"コサイン類似度 = {dot} / ({norm1:.4f} × {norm2:.4f}) = {cosine_similarity(user1, user2):.4f}")
print()

# 全ペアの類似度
print("--- 全ユーザー間の類似度 ---")
print(f"ユーザー1 vs ユーザー2: {cosine_similarity(user1, user2):.4f}")
print(f"ユーザー1 vs ユーザー3: {cosine_similarity(user1, user3):.4f}")
print(f"ユーザー2 vs ユーザー3: {cosine_similarity(user2, user3):.4f}")
print()
print("→ ユーザー1と2は似た好み、ユーザー3は異なる好みを持っている")

---
# まとめ

## Part 1で学んだこと

1. **cos θ** は2つの方向の類似性を -1 から 1 の値で表す
2. **ベクトルの内積** は各要素の積の和: $\vec{a} \cdot \vec{b} = \sum a_i b_i$
3. **コサイン類似度** は内積を正規化したもの:

$$\text{cosine\_similarity}(\vec{a}, \vec{b}) = \frac{\vec{a} \cdot \vec{b}}{\|\vec{a}\| \|\vec{b}\|}$$

## レコメンドシステムでの活用

- ユーザーの評価履歴をベクトルとして表現
- コサイン類似度でユーザー間の類似性を計算
- 類似ユーザーが高評価した商品を推薦

## 次のステップ（Part 2以降）

- 協調フィルタリングの実装
- 行列因子分解
- 勾配降下法による最適化